In [23]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Processor
import soundfile as sf
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from torch import nn
import soundfile as sf


In [24]:

class AudioDataset(Dataset):
    def __init__(self, root_dir, processor):
        """
        root_dir: Ses dosyalarının bulunduğu kök dizin.
        processor: Wav2Vec2Processor nesnesi.
        """
        self.processor = processor
        self.samples = []

        for label in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label)
            if not os.path.isdir(class_dir):
                continue
            for fname in os.listdir(class_dir):
                if fname.lower().endswith('.wav'):
                    self.samples.append((os.path.join(class_dir, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        speech, _ = sf.read(file_path)
        input_values = self.processor(speech, return_tensors="pt", sampling_rate=16000).input_values
        input_values = input_values.squeeze(0)  # Batch boyutunu kaldır
        return input_values, torch.tensor(int(label == "AI COVER"))


In [25]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
class AudioClassifier(nn.Module):
    def __init__(self, num_labels):
        super(AudioClassifier, self).__init__()
        self.wav2vec2 = model
        self.classifier = nn.Linear(model.config.hidden_size, num_labels)

def forward(self, input_values):
    # Stereo kanalları varsa ve bunları mono'ya çevirmek istiyorsak,
    # Kanalların ortalamasını alabiliriz. Bu, stereo -> mono dönüşümü için basit bir yöntemdir.
    # Örneğin, input_values boyutu: [batch_size, sequence_length, channels]
    if input_values.dim() == 3 and input_values.size(-1) == 2:
        input_values = input_values.mean(dim=-1)
    
    # Alternatif olarak, sadece bir kanalı seçmek istiyorsanız:
    # input_values = input_values[..., 0] # İlk kanalı seç
    
    with torch.no_grad():
        # Wav2Vec2.0 modeli (feature extractor) ile özellik çıkarımı
        # Burada input_values boyutu: [batch_size, sequence_length]
        features = self.wav2vec2(input_values).last_hidden_state.mean(dim=1)
    
    # Sınıflandırıcı katmana özellikleri besle
    return self.classifier(features)



Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Veri setini ve veri yükleyiciyi oluşturma
root_dir = '/mnt/d/GITHUB/AbdullahAliGUN/AI-Cover-Classification/Data'  # Kök dizin yolu
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
dataset = AudioDataset(root_dir, processor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Bu bölümde, dataloader üzerinden iterasyon yaparak eğitim yapabilirsiniz


In [27]:
# Model eğitimi
def train(model, dataloader, loss_fn, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (input_values, labels) in enumerate(dataloader):
            optimizer.zero_grad()
            logits = model(input_values)
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}")


In [28]:
# Model ve optimizasyon
model = AudioClassifier(num_labels=2)  # Yukarıda tanımlanan AudioClassifier sınıfını kullanın
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Model eğitimi
train(model, dataloader, loss_fn, optimizer, epochs=10)

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [2, 1, 1323000, 2]